# Pipelines

In [1]:
%run "./Getting started.ipynb"
import requests
base_url = 'http://localhost:6000'
while True:
    try:
        r=requests.get(base_url)
        if r.text == 'EMERGENT API':
            break
    except:
        continue
print(r.text)

C:\emergent\emergent
Overwriting C:\emergent/emergent/networks/test/things/test_thing.py
Overwriting C:\emergent/emergent/networks/test/hubs/test_hub.py
Overwriting C:\emergent/emergent/networks/test/network.py
DataDict([('thing', {'X': None, 'Y': None})])
New state: DataDict([('thing', {'X': 1, 'Y': 2})])
 * Serving Flask app "emergent.API.API" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
EMERGENT API


The Pipeline class allows us to combine multiple optimization processes into a single process. For example, we could do a coarse grid search to obtain an initial signal, then shrink the bounds to enclose the signal and sample with higher resolution, then finally fit all observed data to a model which predicts the location of the maximum:

In [10]:
import numpy as np
from emergent.pipeline import Pipeline, Source, GridSearch, GaussianModel, Rescale, DifferentialEvolution
from emergent.utilities.containers import DataDict

''' Define data source '''
hub = network.hubs['hub']
hub.range = DataDict({'thing': {'X': {'min': 0, 'max': 1}, 'Y': {'min': 0, 'max': 1}}})
thing = hub.children['thing']
state = {'thing': thing.state}
bounds = hub.range.copy()
experiment = hub.gaussian
params = {'sigma_x': 0.3, 'sigma_y': 0.8, 'x0': 0.3, 'y0': 0.6, 'noise':0.0}
source = Source(state, bounds, experiment, params)

''' Declare and run pipeline.  '''
pipe = Pipeline(state, source)

pipe.add(GridSearch(params={'Steps': 5}))
pipe.add(Rescale(threshold=0.5))
pipe.add(GridSearch(params={'Steps':30}))
# pipe.add(DifferentialEvolution())
pipe.add(GaussianModel(optimizer = DifferentialEvolution()))

# pipe.add(GaussianModel(optimizer = GridSearch(params={'Steps': 30})))

points, costs = pipe.run()

print(points[-1], costs[-1])
print('done')


[0.30172414 0.5862069 ] -0.9996697599184603
done


In [3]:
pipe.points

array([[0.        , 0.        ],
       [0.        , 0.25      ],
       [0.        , 0.5       ],
       [0.        , 0.75      ],
       [0.        , 1.        ],
       [0.25      , 0.        ],
       [0.25      , 0.25      ],
       [0.25      , 0.5       ],
       [0.25      , 0.75      ],
       [0.25      , 1.        ],
       [0.5       , 0.        ],
       [0.5       , 0.25      ],
       [0.5       , 0.5       ],
       [0.5       , 0.75      ],
       [0.5       , 1.        ],
       [0.75      , 0.        ],
       [0.75      , 0.25      ],
       [0.75      , 0.5       ],
       [0.75      , 0.75      ],
       [0.75      , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.25      ],
       [1.        , 0.5       ],
       [1.        , 0.75      ],
       [1.        , 1.        ],
       [0.29999999, 0.60000002],
       [0.29999999, 0.60000002]])